# Reading Data from MongoDB via PyMongo

In [11]:
import json
from pymongo import MongoClient

with open("secrets.json", "r") as f:
    secrets = json.load(f)
    MONGODB_USERNAME = secrets["PYMONGO_USERNAME"]
    MONGODB_PASSWORD = secrets["PYMONGO_PASSWORD"]

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

cluster_uri = "sparks-materials-inform.bgydt"
client = MongoClient(
    f"mongodb+srv://{MONGODB_USERNAME}:{MONGODB_PASSWORD}@{cluster_uri}.mongodb.net/?retryWrites=true&w=majority"
)

database_name = "clslab-light-mixing"
collection_name = "hackathon"

In [12]:
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    raise e

Pinged your deployment. You successfully connected to MongoDB!


In [40]:
from tqdm.notebook import tqdm

db = client[database_name]
collection = db[collection_name]

pattern = r"^benchmark-dev-[0-9a-z]{4}$"
""" Explanation of regex pattern:
^ asserts the start of the string.
benchmark-dev- matches the literal characters "benchmark-dev-".
[0-9a-z] matches any digit (0-9) or lowercase letter (a-z).
{4} specifies that the previous character set should be repeated exactly 4 times.
$ asserts the end of the string.
"""

filter = {
    "encrypted_device_id_truncated": "test",
    "background": {"$exists": True},
    "_input_message._session_id": {"$regex": pattern},
}
num_docs = collection.count_documents(filter, maxTimeMS=60000)
print(num_docs)

41131


In [41]:
cursor = collection.find(filter)
results = []
# gets progressively faster as the cursor progresses
for result in tqdm(cursor, total=num_docs):
    results.append(result)

  0%|          | 0/41131 [00:00<?, ?it/s]

In [83]:
import pandas as pd
df = pd.DataFrame(results).set_index("_id")
df.head(1)

,utc_timestamp,background,ch470,_input_message,ch410,ch440,sd_card_ready,ch510,ch550,ch670,utc_time_str,onboard_temperature_K,encrypted_device_id_truncated,ch620,device_nickname,ch583
_id,,,,,,,,,,,,,,,,
63a51bddb64ebdefe7c7b708,1671764950,"{'ch583': 15, 'ch670': 22, 'ch510': 39, 'ch410...",2189,"{'_session_id': 'benchmark-dev-001a', 'B': 87,...",125,2470,False,929,210,188,2022-12-23 03:09:10,294.1085,test,2108,CLSLab-light-public-test,1025


In [84]:
inp_msg_df = pd.json_normalize(df["_input_message"]).set_index(df.index)
background_df = pd.json_normalize(df["background"]).set_index(df.index)
background_df = background_df.add_suffix("_bg")
df.drop(columns=["_input_message", "background"], inplace=True)
cat_df = pd.concat([df, inp_msg_df, background_df], axis=1)
cat_df.shape

(41131, 32)

In [85]:
# find columns that match ch###
channel_names = cat_df.filter(regex="^ch[0-9]{3}$").columns
channel_names

Index(['ch470', 'ch410', 'ch440', 'ch510', 'ch550', 'ch670', 'ch620', 'ch583'], dtype='object')

In [87]:
# subtract the background from the signal and assign to new variable
for ch in channel_names:
    cat_df[f"{ch}_sub"] = cat_df[ch] - cat_df[f"{ch}_bg"]    

In [97]:
cat_df.head(1).T

_id,63a51bddb64ebdefe7c7b708
utc_timestamp,1671764950
ch470,2189
ch410,125
ch440,2470
sd_card_ready,False
ch510,929
ch550,210
ch670,188
utc_time_str,2022-12-23 03:09:10
onboard_temperature_K,294.1085


In [98]:
target_df = cat_df.filter(regex="^ch[0-9]{3}_sub$").copy()

In [100]:
feature_df = cat_df[["R", "G", "B", "atime", "astep", "gain", "onboard_temperature_K"]]

In [102]:
data_df = pd.concat([feature_df, target_df], axis=1)
data_df.to_csv(f"../data/external/{database_name}-{collection_name}.csv")
data_df.columns

Index(['R', 'G', 'B', 'atime', 'astep', 'gain', 'onboard_temperature_K',
       'ch470_sub', 'ch410_sub', 'ch440_sub', 'ch510_sub', 'ch550_sub',
       'ch670_sub', 'ch620_sub', 'ch583_sub'],
      dtype='object')

## Code Graveyard

In [ ]:
# results = collection.aggregate([{"$match": {"fba": {"$exists": True}}}])
# filter = {"scores": {"$exists": True}}
    # "_input_message._session_id": "3b687a62-6144-4d90-811b-b41fb7028c44",

In [ ]:
# import json
# print(df.columns)
# inp_msg = pd.json_normalize(df["_input_message"])

# counts = inp_msg["_session_id"].value_counts()
# counts[counts > 100]

In [ ]:
# df["scores"] = df["scores"].apply(json.dumps)

In [ ]:
# bg_channel_names = cat_df.filter(regex="^ch[0-9]{3}_bg$").columns
# bg_channel_names

In [ ]:
# df.drop(columns=["_input_message", "background"], inplace=True)

In [ ]:
# ch_cols = [col for col in cat_df.columns if col.startswith("ch")]